![xarray Logo](http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png "xarray Logo")

# Introduction to Xarray

---

In [1]:
from datetime import timedelta

import numpy as np
import pandas as pd
import xarray as xr

## Introducing the `DataArray` and `Dataset`

Xarray expands on the capabilities on NumPy arrays, providing a lot of streamlined data manipulation. It is similar in that respect to Pandas, but whereas Pandas excels at working with tabular data, Xarray is focused on N-dimensional arrays of data (i.e. grids). Its interface is based largely on the netCDF data model (variables, attributes, and dimensions), but it goes beyond the traditional netCDF interfaces to provide functionality similar to netCDF-java's [Common Data Model (CDM)](https://docs.unidata.ucar.edu/netcdf-java/current/userguide/common_data_model_overview.html). 

<center> <img src="img/dataset-diagram.png" width="700"/> </center>

### Creation of a `DataArray` object

The `DataArray` is one of the basic building blocks of Xarray (see docs [here](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataarray)). It provides a `numpy.ndarray`-like object that expands to provide two critical pieces of functionality:

1. Coordinate names and values are stored with the data, making slicing and indexing much more powerful
2. It has a built-in container for attributes

Here we'll initialize a `DataArray` object by wrapping a plain NumPy array, and explore a few of its properties.

#### Generate a random numpy array

For our first example, we'll just create a random array of "temperature" data in units of Kelvin:

In [2]:
data = 283 + 5 * np.random.randn(5, 3, 4)
data

array([[[271.65941635, 286.37134725, 278.86567113, 282.88490612],
        [285.75198216, 279.14746756, 285.64955161, 275.77564201],
        [284.04960634, 288.30987099, 282.44057489, 279.94656876]],

       [[285.02185096, 280.47067994, 278.02290479, 284.00594038],
        [286.64168251, 285.75213086, 285.78921707, 291.87282785],
        [291.54235292, 280.80824102, 282.19679708, 281.35512062]],

       [[294.44840701, 287.43893366, 284.63734977, 283.01140651],
        [279.90959222, 277.03664829, 284.88789667, 288.5085483 ],
        [284.81018062, 282.14857395, 278.26829421, 274.2838187 ]],

       [[279.99654911, 272.30242432, 280.21214344, 284.6122661 ],
        [286.84517346, 291.83905312, 277.90115662, 282.40965278],
        [285.51646663, 290.92861044, 283.55279685, 287.40765717]],

       [[283.87178406, 278.69118788, 277.44830667, 287.47626436],
        [275.65567311, 284.62488837, 279.31043631, 278.57236551],
        [290.32220797, 279.96125017, 287.36719824, 286.20727853]]])

#### Wrap the array: first attempt

Now we create a basic `DataArray` just by passing our plain `data` as input:

In [3]:
temp = xr.DataArray(data)
temp

<xarray.DataArray (dim_0: 5, dim_1: 3, dim_2: 4)>
array([[[271.65941635, 286.37134725, 278.86567113, 282.88490612],
        [285.75198216, 279.14746756, 285.64955161, 275.77564201],
        [284.04960634, 288.30987099, 282.44057489, 279.94656876]],

       [[285.02185096, 280.47067994, 278.02290479, 284.00594038],
        [286.64168251, 285.75213086, 285.78921707, 291.87282785],
        [291.54235292, 280.80824102, 282.19679708, 281.35512062]],

       [[294.44840701, 287.43893366, 284.63734977, 283.01140651],
        [279.90959222, 277.03664829, 284.88789667, 288.5085483 ],
        [284.81018062, 282.14857395, 278.26829421, 274.2838187 ]],

       [[279.99654911, 272.30242432, 280.21214344, 284.6122661 ],
        [286.84517346, 291.83905312, 277.90115662, 282.40965278],
        [285.51646663, 290.92861044, 283.55279685, 287.40765717]],

       [[283.87178406, 278.69118788, 277.44830667, 287.47626436],
        [275.65567311, 284.62488837, 279.31043631, 278.57236551],
        [290.32220797, 279.96125017, 287.36719824, 286.20727853]]])
Dimensions without coordinates: dim_0, dim_1, dim_2

Note two things:

1. Xarray generates some basic dimension names for us (`dim_0`, `dim_1`, `dim_2`). We'll improve this with better names in the next example.
2. Wrapping the numpy array in a `DataArray` gives us a rich display in the notebook! (Try clicking the array symbol to expand or collapse the view)

#### Assign dimension names

Much of the power of Xarray comes from making use of named dimensions. So let's add some more useful names! We can do that by passing an ordered list of names using the keyword argument `dims`:

In [4]:
temp = xr.DataArray(data, dims=['time', 'lat', 'lon'])
temp.lat

<xarray.DataArray 'lat' (lat: 3)>
array([0, 1, 2])
Dimensions without coordinates: lat

This is already improved upon from a NumPy array, because we have names for each of the dimensions (or axes in NumPy parlance). Even better, we can take arrays representing the values for the coordinates for each of these dimensions and associate them with the data when we create the `DataArray`. We'll see this in the next example.

### Create a `DataArray` with named Coordinates

#### Make time and space coordinates

Here we will use [Pandas](../pandas) to create an array of [datetime data](../datetime), which we will then use to create a `DataArray` with a named coordinate `time`.

In [5]:
times = pd.date_range('2018-01-01', periods=5)
times

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05'],
              dtype='datetime64[ns]', freq='D')

We'll also create arrays to represent sample longitude and latitude:

In [6]:
lons = np.linspace(-120, -60, 4)
lats = np.linspace(25, 55, 3)

#### Initialize the `DataArray` with complete coordinate info

When we create the `DataArray` instance, we pass in the arrays we just created:

In [7]:
temp = xr.DataArray(data, 
                    coords=[times, lats, lons], 
                    dims=['time', 'lat', 'lon'])
temp

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[271.65941635, 286.37134725, 278.86567113, 282.88490612],
        [285.75198216, 279.14746756, 285.64955161, 275.77564201],
        [284.04960634, 288.30987099, 282.44057489, 279.94656876]],

       [[285.02185096, 280.47067994, 278.02290479, 284.00594038],
        [286.64168251, 285.75213086, 285.78921707, 291.87282785],
        [291.54235292, 280.80824102, 282.19679708, 281.35512062]],

       [[294.44840701, 287.43893366, 284.63734977, 283.01140651],
        [279.90959222, 277.03664829, 284.88789667, 288.5085483 ],
        [284.81018062, 282.14857395, 278.26829421, 274.2838187 ]],

       [[279.99654911, 272.30242432, 280.21214344, 284.6122661 ],
        [286.84517346, 291.83905312, 277.90115662, 282.40965278],
        [285.51646663, 290.92861044, 283.55279685, 287.40765717]],

       [[283.87178406, 278.69118788, 277.44830667, 287.47626436],
        [275.65567311, 284.62488837, 279.31043631, 278.57236551],
        [290.32220797, 279.96125017, 287.36719824, 286.20727853]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0

#### Set useful attributes

...and while we're at it, we can also set some attribute metadata:

In [8]:
temp.attrs['units'] = 'kelvin'
temp.attrs['standard_name'] = 'air_temperature'

temp

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[271.65941635, 286.37134725, 278.86567113, 282.88490612],
        [285.75198216, 279.14746756, 285.64955161, 275.77564201],
        [284.04960634, 288.30987099, 282.44057489, 279.94656876]],

       [[285.02185096, 280.47067994, 278.02290479, 284.00594038],
        [286.64168251, 285.75213086, 285.78921707, 291.87282785],
        [291.54235292, 280.80824102, 282.19679708, 281.35512062]],

       [[294.44840701, 287.43893366, 284.63734977, 283.01140651],
        [279.90959222, 277.03664829, 284.88789667, 288.5085483 ],
        [284.81018062, 282.14857395, 278.26829421, 274.2838187 ]],

       [[279.99654911, 272.30242432, 280.21214344, 284.6122661 ],
        [286.84517346, 291.83905312, 277.90115662, 282.40965278],
        [285.51646663, 290.92861044, 283.55279685, 287.40765717]],

       [[283.87178406, 278.69118788, 277.44830667, 287.47626436],
        [275.65567311, 284.62488837, 279.31043631, 278.57236551],
        [290.32220797, 279.96125017, 287.36719824, 286.20727853]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

#### Attributes are not preserved by default!

Notice what happens if we perform a mathematical operaton with the `DataArray`: the coordinate values persist, but the attributes are lost. This is done because it is very challenging to know if the attribute metadata is still correct or appropriate after arbitrary arithmetic operations.

To illustrate this, we'll do a simple unit conversion from Kelvin to Celsius:

In [9]:
temp_in_celsius = temp - 273.15
temp_in_celsius

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[-1.49058365, 13.22134725,  5.71567113,  9.73490612],
        [12.60198216,  5.99746756, 12.49955161,  2.62564201],
        [10.89960634, 15.15987099,  9.29057489,  6.79656876]],

       [[11.87185096,  7.32067994,  4.87290479, 10.85594038],
        [13.49168251, 12.60213086, 12.63921707, 18.72282785],
        [18.39235292,  7.65824102,  9.04679708,  8.20512062]],

       [[21.29840701, 14.28893366, 11.48734977,  9.86140651],
        [ 6.75959222,  3.88664829, 11.73789667, 15.3585483 ],
        [11.66018062,  8.99857395,  5.11829421,  1.1338187 ]],

       [[ 6.84654911, -0.84757568,  7.06214344, 11.4622661 ],
        [13.69517346, 18.68905312,  4.75115662,  9.25965278],
        [12.36646663, 17.77861044, 10.40279685, 14.25765717]],

       [[10.72178406,  5.54118788,  4.29830667, 14.32626436],
        [ 2.50567311, 11.47488837,  6.16043631,  5.42236551],
        [17.17220797,  6.81125017, 14.21719824, 13.05727853]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0

For an in-depth discussion of how Xarray handles metadata, start in the Xarray docs [here](http://xarray.pydata.org/en/stable/getting-started-guide/faq.html#approach-to-metadata).

### The `Dataset`: a container for `DataArray`s with shared coordinates

Along with `DataArray`, the other key object type in Xarray is the `Dataset`: a dictionary-like container that holds one or more `DataArray`s, which can also optionally share coordinates (see docs [here](http://xarray.pydata.org/en/stable/user-guide/data-structures.html#dataset)).

The most common way to create a `Dataset` object is to load data from a file (see [below](#Opening-netCDF-data)). Here, instead, we will create another `DataArray` and combine it with our `temp` data.

This will illustrate how the information about common coordinate axes is used.

#### Create a pressure `DataArray` using the same coordinates

This code mirrors how we created the `temp` object above.

In [10]:
pressure_data = 1000.0 + 5 * np.random.randn(5, 3, 4)
pressure = xr.DataArray(
    pressure_data, coords=[times, lats, lons], dims=['time', 'lat', 'lon']
)
pressure.attrs['units'] = 'hPa'
pressure.attrs['standard_name'] = 'air_pressure'

pressure

<xarray.DataArray (time: 5, lat: 3, lon: 4)>
array([[[1001.57634645,  999.70679367, 1006.41343594,  994.37965094],
        [1006.96778425,  999.13042912,  993.72380375, 1006.67826436],
        [1007.70530764,  997.56843754, 1005.55054789,  995.19389422]],

       [[ 997.11464554, 1004.81901564,  996.17349534,  999.06066136],
        [1006.45732562,  997.4927907 ,  999.65070932, 1003.74606132],
        [1009.19844773,  990.62658534, 1006.49867061, 1005.79062417]],

       [[ 995.94833433, 1003.54954768,  995.42199598,  999.37976641],
        [ 994.93510243, 1003.77979699, 1006.390009  , 1012.85853476],
        [1001.59016538,  999.25400849, 1002.81925873, 1005.58449313]],

       [[ 998.19933449,  999.73396371, 1004.40600103,  998.91648942],
        [ 996.6811946 ,  996.86832917, 1005.74906168,  994.31870244],
        [ 996.93459937, 1006.59598109, 1005.20121629,  999.08065765]],

       [[ 993.33336731, 1010.1025046 ,  996.48038271, 1002.43904812],
        [1000.41141811, 1005.20807031, 1003.89420434, 1005.61137953],
        [1009.30980057, 1008.71541804, 1002.01849874, 1007.37625125]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

#### Create a `Dataset` object

Each `DataArray` in our `Dataset` needs a name! 

The most straightforward way to create a `Dataset` with our `temp` and `pressure` arrays is to pass a dictionary using the keyword argument `data_vars`:

In [11]:
ds = xr.Dataset(data_vars={'Temperature': temp, 'Pressure': pressure})
ds

<xarray.Dataset>
Dimensions:      (time: 5, lat: 3, lon: 4)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat          (lat) float64 25.0 40.0 55.0
  * lon          (lon) float64 -120.0 -100.0 -80.0 -60.0
Data variables:
    Temperature  (time, lat, lon) float64 271.7 286.4 278.9 ... 287.4 286.2
    Pressure     (time, lat, lon) float64 1.002e+03 999.7 ... 1.007e+03

Notice that the `Dataset` object `ds` is aware that both data arrays sit on the same coordinate axes.

#### Access Data variables and Coordinates in a `Dataset`

We can pull out any of the individual `DataArray` objects in a few different ways.

Using the "dot" notation:

In [12]:
ds.Pressure

<xarray.DataArray 'Pressure' (time: 5, lat: 3, lon: 4)>
array([[[1001.57634645,  999.70679367, 1006.41343594,  994.37965094],
        [1006.96778425,  999.13042912,  993.72380375, 1006.67826436],
        [1007.70530764,  997.56843754, 1005.55054789,  995.19389422]],

       [[ 997.11464554, 1004.81901564,  996.17349534,  999.06066136],
        [1006.45732562,  997.4927907 ,  999.65070932, 1003.74606132],
        [1009.19844773,  990.62658534, 1006.49867061, 1005.79062417]],

       [[ 995.94833433, 1003.54954768,  995.42199598,  999.37976641],
        [ 994.93510243, 1003.77979699, 1006.390009  , 1012.85853476],
        [1001.59016538,  999.25400849, 1002.81925873, 1005.58449313]],

       [[ 998.19933449,  999.73396371, 1004.40600103,  998.91648942],
        [ 996.6811946 ,  996.86832917, 1005.74906168,  994.31870244],
        [ 996.93459937, 1006.59598109, 1005.20121629,  999.08065765]],

       [[ 993.33336731, 1010.1025046 ,  996.48038271, 1002.43904812],
        [1000.41141811, 1005.20807031, 1003.89420434, 1005.61137953],
        [1009.30980057, 1008.71541804, 1002.01849874, 1007.37625125]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

... or using dictionary access like this:

In [13]:
ds['Pressure']

<xarray.DataArray 'Pressure' (time: 5, lat: 3, lon: 4)>
array([[[1001.57634645,  999.70679367, 1006.41343594,  994.37965094],
        [1006.96778425,  999.13042912,  993.72380375, 1006.67826436],
        [1007.70530764,  997.56843754, 1005.55054789,  995.19389422]],

       [[ 997.11464554, 1004.81901564,  996.17349534,  999.06066136],
        [1006.45732562,  997.4927907 ,  999.65070932, 1003.74606132],
        [1009.19844773,  990.62658534, 1006.49867061, 1005.79062417]],

       [[ 995.94833433, 1003.54954768,  995.42199598,  999.37976641],
        [ 994.93510243, 1003.77979699, 1006.390009  , 1012.85853476],
        [1001.59016538,  999.25400849, 1002.81925873, 1005.58449313]],

       [[ 998.19933449,  999.73396371, 1004.40600103,  998.91648942],
        [ 996.6811946 ,  996.86832917, 1005.74906168,  994.31870244],
        [ 996.93459937, 1006.59598109, 1005.20121629,  999.08065765]],

       [[ 993.33336731, 1010.1025046 ,  996.48038271, 1002.43904812],
        [1000.41141811, 1005.20807031, 1003.89420434, 1005.61137953],
        [1009.30980057, 1008.71541804, 1002.01849874, 1007.37625125]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          hPa
    standard_name:  air_pressure

We'll return to the `Dataset` object when we start loading data from files.

## Subsetting and selection by coordinate values

Much of the power of labeled coordinates comes from the ability to select data based on coordinate names and values, rather than array indices. We'll explore this briefly here.

### NumPy-like selection

Suppose we want to extract all the spatial data for one single date: January 2, 2018. It's possible to achieve that with NumPy-like index selection:

In [14]:
indexed_selection = temp[1, :, :]  # Index 1 along axis 0 is the time slice we want...
indexed_selection

<xarray.DataArray (lat: 3, lon: 4)>
array([[285.02185096, 280.47067994, 278.02290479, 284.00594038],
       [286.64168251, 285.75213086, 285.78921707, 291.87282785],
       [291.54235292, 280.80824102, 282.19679708, 281.35512062]])
Coordinates:
    time     datetime64[ns] 2018-01-02
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

HOWEVER, notice that this requires us (the user / programmer) to have **detailed knowledge** of the order of the axes and the meaning of the indices along those axes!

_**Named coordinates free us from this burden...**_

### Selecting with `.sel()`

We can instead select data based on coordinate values using the `.sel()` method, which takes one or more named coordinate(s) as keyword argument:

In [15]:
named_selection = temp.sel(time='2018-01-02')
named_selection

<xarray.DataArray (lat: 3, lon: 4)>
array([[285.02185096, 280.47067994, 278.02290479, 284.00594038],
       [286.64168251, 285.75213086, 285.78921707, 291.87282785],
       [291.54235292, 280.80824102, 282.19679708, 281.35512062]])
Coordinates:
    time     datetime64[ns] 2018-01-02
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

We got the same result, but 
- we didn't have to know anything about how the array was created or stored
- our code is agnostic about how many dimensions we are dealing with
- the intended meaning of our code is much clearer!

### Approximate selection and interpolation

With time and space data, we frequently want to sample "near" the coordinate points in our dataset. Here are a few simple ways to achieve that.

#### Nearest-neighbor sampling

Suppose we want to sample the nearest datapoint within 2 days of date `2018-01-07`. Since the last day on our `time` axis is `2018-01-05`, this is well-posed.

`.sel` has the flexibility to perform nearest neighbor sampling, taking an optional tolerance:

In [16]:
temp.sel(time='2018-01-07', method='nearest', tolerance=timedelta(days=2))

<xarray.DataArray (lat: 3, lon: 4)>
array([[283.87178406, 278.69118788, 277.44830667, 287.47626436],
       [275.65567311, 284.62488837, 279.31043631, 278.57236551],
       [290.32220797, 279.96125017, 287.36719824, 286.20727853]])
Coordinates:
    time     datetime64[ns] 2018-01-05
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

where we see that `.sel` indeed pulled out the data for date `2018-01-05`.

#### Interpolation

The `.interp()` method (see the docs [here](http://xarray.pydata.org/en/stable/interpolation.html)) works similarly to `.sel()`. Using `.interp()`, we can interpolate to any latitude/longitude location:

In [17]:
temp.interp(lon=-105, lat=40)

<xarray.DataArray (time: 5)>
array([280.79859621, 285.97451878, 277.75488427, 290.59058321,
       282.38258455])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
    lon      int64 -105
    lat      int64 40
Attributes:
    units:          kelvin
    standard_name:  air_temperature

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Info</p>
    Xarray's interpolation functionality requires the <a href="https://scipy.org/">SciPy</a> package!
</div>

### Slicing along coordinates

Frequently we want to select a range (or _slice_) along one or more coordinate(s). We can achieve this by passing a Python [slice](https://docs.python.org/3/library/functions.html#slice) object to `.sel()`, as follows:

In [18]:
temp.sel(
    time=slice('2018-01-01', '2018-01-03'), lon=slice(-110, -70), lat=slice(25, 45)
)

<xarray.DataArray (time: 3, lat: 2, lon: 2)>
array([[[286.37134725, 278.86567113],
        [279.14746756, 285.64955161]],

       [[280.47067994, 278.02290479],
        [285.75213086, 285.78921707]],

       [[287.43893366, 284.63734977],
        [277.03664829, 284.88789667]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 2018-01-03
  * lat      (lat) float64 25.0 40.0
  * lon      (lon) float64 -100.0 -80.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Info</p>
    The calling sequence for <code>slice</code> always looks like <code>slice(start, stop[, step])</code>, where <code>step</code> is optional.
</div>

Notice how the length of each coordinate axis has changed due to our slicing.

### One more selection method: `.loc`

All of these operations can also be done within square brackets on the `.loc` attribute of the `DataArray`:


In [19]:
temp.loc['2018-01-02']

<xarray.DataArray (lat: 3, lon: 4)>
array([[285.02185096, 280.47067994, 278.02290479, 284.00594038],
       [286.64168251, 285.75213086, 285.78921707, 291.87282785],
       [291.54235292, 280.80824102, 282.19679708, 281.35512062]])
Coordinates:
    time     datetime64[ns] 2018-01-02
  * lat      (lat) float64 25.0 40.0 55.0
  * lon      (lon) float64 -120.0 -100.0 -80.0 -60.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

This is sort of in between the NumPy-style selection
```
temp[1,:,:]
```
and the fully label-based selection using `.sel()`

With `.loc`, we make use of the coordinate *values*, but lose the ability to specify the *names* of the various dimensions. Instead, the slicing must be done in the correct order:

In [20]:
temp.loc['2018-01-01':'2018-01-03', 25:45, -110:-70]

<xarray.DataArray (time: 3, lat: 2, lon: 2)>
array([[[286.37134725, 278.86567113],
        [279.14746756, 285.64955161]],

       [[280.47067994, 278.02290479],
        [285.75213086, 285.78921707]],

       [[287.43893366, 284.63734977],
        [277.03664829, 284.88789667]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 2018-01-03
  * lat      (lat) float64 25.0 40.0
  * lon      (lon) float64 -100.0 -80.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

One advantage of using `.loc` is that we can use NumPy-style slice notation like `25:45`, rather than the more verbose `slice(25,45)`. But of course that also works:

In [21]:
temp.loc['2018-01-01':'2018-01-03', slice(25, 45), -110:-70]

<xarray.DataArray (time: 3, lat: 2, lon: 2)>
array([[[286.37134725, 278.86567113],
        [279.14746756, 285.64955161]],

       [[280.47067994, 278.02290479],
        [285.75213086, 285.78921707]],

       [[287.43893366, 284.63734977],
        [277.03664829, 284.88789667]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 2018-01-03
  * lat      (lat) float64 25.0 40.0
  * lon      (lon) float64 -100.0 -80.0
Attributes:
    units:          kelvin
    standard_name:  air_temperature

What *doesn't* work is passing the slices in a different order:

In [22]:
# This will generate an error
# temp.loc[-110:-70, 25:45,'2018-01-01':'2018-01-03']

# Opening data

With its close ties to the netCDF data model, Xarray also supports netCDF as a first-class file format. This means it has easy support for opening netCDF datasets, so long as they conform to some of Xarray's limitations (such as 1-dimensional coordinates).

### Access netCDF data with `xr.open_dataset`

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Info</p>
    Here we're getting the data from Project Pythia's custom library of example data, which we already imported above with <code>from pythia_datasets import DATASETS</code>. The <code>DATASETS.fetch()</code> method will automatically download and cache our example data file <code>NARR_19930313_0000.nc</code> locally.
</div>

In [23]:
#filepath = DATASETS.fetch('NARR_19930313_0000.nc')

Once we have a valid path to a data file that Xarray knows how to read, we can open it like this:

### Open / Load 

currents_liguria --> folder
currents_01012010.nc
currents_02012010.nc
currents_03012010.nc

dir_data = Path('currents_liguria')

xr.open_dataset(dir_data / 'currents_01012010.nc')  
xr.load_dataset(dir_data / 'currents_01012010.nc')

xr.open_dataarray(dir_data / 'uo_currents_01012010.nc')

xr.open_mfdataset(dir_data.glob('currents_*.nc'))
xr.open_mfdataset(dir_data.glob('currents_*2010.nc'))

xr.open_mfdataset('XXX*.grb2', engine='cfgrib')



In [25]:
ds = xr.open_mfdataset('data/WW3_mediterr_*.grb2', engine='cfgrib')
ds

/opt/anaconda3/lib/python3.8/site-packages/xarray/backends/plugins.py:71: RuntimeWarning: Engine 'cfgrib' loading failed:
ecCodes library not found using ['eccodes', 'libeccodes.so', 'libeccodes']
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


<xarray.Dataset>
Dimensions:    (lev: 33, d2: 2, lat: 180, lon: 360)
Coordinates:
  * lev        (lev) float64 0.0 10.0 20.0 30.0 ... 4e+03 4.5e+03 5e+03 5.5e+03
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
Dimensions without coordinates: d2
Data variables:
    lev_bnds   (lev, d2) float64 dask.array<chunksize=(33, 2), meta=np.ndarray>
    lat_bnds   (lat, d2) float64 dask.array<chunksize=(180, 2), meta=np.ndarray>
    lon_bnds   (lon, d2) float64 dask.array<chunksize=(360, 2), meta=np.ndarray>
    areacello  (lat, lon) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
Attributes: (12/44)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   674885.0
    branch_time_in_parent:  219000.0
    case_id:                972
    ...                     ...
    sub_experiment_id:      none
    table_id:               Ofx
    tracking_id:            hdl:21.14100/bca86935-7746-42d9-a7af-fe5607809cc4
    variable_id:            areacello
    variant_info:           CMIP6 20th century experiments (1850-2014) with C...
    variant_label:          r11i1p1f1

## Subsetting the `Dataset`

Our call to `xr.open_dataset()` above returned a `Dataset` object that we've decided to call `ds`. We can then pull out individual fields:

In [27]:
ds = xr.open_dataset('data/NARR_19930313_0000.nc')

ds.isobaric1

<xarray.DataArray 'areacello' (lat: 180, lon: 360)>
dask.array<open_dataset-a9d57b593003627d31d20c97e8ea3092areacello, shape=(180, 360), dtype=float64, chunksize=(180, 360), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
Attributes: (12/17)
    cell_methods:   area: sum
    comment:        TAREA
    description:    Cell areas for any grid used to report ocean variables an...
    frequency:      fx
    id:             areacello
    long_name:      Grid-Cell Area for Ocean Variables
    ...             ...
    time_label:     None
    time_title:     No temporal dimensions ... fixed field
    title:          Grid-Cell Area for Ocean Variables
    type:           real
    units:          m2
    variable_id:    areacello

(recall that we can also use dictionary syntax like `ds['isobaric1']` to do the same thing)

`Dataset`s also support much of the same subsetting operations as `DataArray`, but will perform the operation on all data:

In [28]:
ds_1000 = ds.sel(isobaric1=1000.0)
ds_1000

<xarray.Dataset>
Dimensions:    (d2: 2, lat: 180, lon: 360)
Coordinates:
    lev        float64 20.0
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
Dimensions without coordinates: d2
Data variables:
    lev_bnds   (d2) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    lat_bnds   (lat, d2) float64 dask.array<chunksize=(180, 2), meta=np.ndarray>
    lon_bnds   (lon, d2) float64 dask.array<chunksize=(360, 2), meta=np.ndarray>
    areacello  (lat, lon) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
Attributes: (12/44)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   674885.0
    branch_time_in_parent:  219000.0
    case_id:                972
    ...                     ...
    sub_experiment_id:      none
    table_id:               Ofx
    tracking_id:            hdl:21.14100/bca86935-7746-42d9-a7af-fe5607809cc4
    variable_id:            areacello
    variant_info:           CMIP6 20th century experiments (1850-2014) with C...
    variant_label:          r11i1p1f1

And further subsetting to a single `DataArray`:

In [ ]:
ds_1000.Temperature_isobaric

### Aggregation operations

Not only can you use the named dimensions for manual slicing and indexing of data, but you can also use it to control aggregation operations, like `std` (standard deviation):

In [ ]:
u_winds = ds['u-component_of_wind_isobaric']
u_winds.std(dim=['x', 'y'])

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Info</p>
    Aggregation methods for Xarray objects operate over the named coordinate dimension(s) specified by keyword argument <code>dim</code>. Compare to NumPy, where aggregations operate over specified numbered <code>axes</code>.
</div>

Using the sample dataset, we can calculate the mean temperature profile (temperature as a function of pressure) over Colorado within this dataset. For this exercise, consider the bounds of Colorado to be:
 * x: -182km to 424km
 * y: -1450km to -990km
    
(37°N to 41°N and 102°W to 109°W projected to Lambert Conformal projection coordinates)

In [ ]:
temps = ds.Temperature_isobaric
co_temps = temps.sel(x=slice(-182, 424), y=slice(-1450, -990))
prof = co_temps.mean(dim=['x', 'y'])
prof

## Plotting with Xarray

Another major benefit of using labeled data structures is that they enable automated plotting with sensible axis labels. 

### Simple visualization with `.plot()`

Much like we saw in [Pandas](../pandas/pandas), Xarray includes an interface to [Matplotlib](../matplotlib) that we can access through the `.plot()` method of every `DataArray`.

For quick and easy data exploration, we can just call `.plot()` without any modifiers:

In [ ]:
prof.plot()

Here Xarray has generated a line plot of the temperature data against the coordinate variable `isobaric`. Also the metadata are used to auto-generate axis labels and units.

### Customizing the plot

As in Pandas, the `.plot()` method is mostly just a wrapper to Matplotlib, so we can customize our plot in familiar ways.

In this air temperature profile example, we would like to make two changes:
- swap the axes so that we have isobaric levels on the y (vertical) axis of the figure
- make pressure decrease upward in the figure, so that up is up

A few keyword arguments to our `.plot()` call will take care of this:

In [ ]:
prof.plot(y="isobaric1", yincrease=False)

### Plotting 2D data

In the example above, the `.plot()` method produced a line plot.

What if we call `.plot()` on a 2D array?

In [ ]:
temps.sel(isobaric1=1000).plot()

Xarray has recognized that the `DataArray` object calling the plot method has two coordinate variables, and generates a 2D plot using the `pcolormesh` method from Matplotlib.

In this case, we are looking at air temperatures on the 1000 hPa isobaric surface over North America. We could of course improve this figure by using [Cartopy](../cartopy) to handle the map projection and geographic features!

---